### Este notebook contem todo o processo de mineração do dataset de casas em King County, EUA

#### Carregamento dos dados

In [1]:
from pysrc.loading import *

In [2]:
dataframe.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
3942,8861500015,20140520T000000,675000,3,2.25,1990,10260,2.0,0,0,...,8,1990,0,1987,0,98052,47.6801,-122.115,1990,10260
18012,8624700015,20141112T000000,640000,4,3.00,2940,5763,1.0,0,0,...,8,1640,1300,1955,0,98108,47.5589,-122.295,2020,7320
8338,6902000100,20140915T000000,500000,3,1.75,2420,65501,2.0,0,1,...,8,2420,0,1984,0,98074,47.6525,-122.087,2970,19036
9894,9433000390,20141218T000000,784950,4,2.75,2840,4227,3.0,0,0,...,9,2840,0,2014,0,98052,47.7097,-122.107,2880,4227
16525,2025059026,20150225T000000,1980000,4,3.50,4500,44384,1.0,0,0,...,12,3340,1160,1990,0,98004,47.6323,-122.192,2540,26287


In [3]:
# Colunas da base de dados, inicialmente
showInitialCols()

21 colunas
['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


## Preprocessamento

In [4]:
from pysrc.preprocessing import *

#### Remover colunas que não nos interessam

As colunas removidas são impressas abaixo

In [5]:
dropUselessCols()

In [6]:
# Colunas restantes
showFinalCols()

16 colunas restantes
Removidas  ['id', 'date', 'zipcode', 'lat', 'long']

 ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']


#### Verificar qualidade dos dados restantes

In [7]:
# Checando premissas fundamentais sobre os atributos
# Valores nao mencionados abaixo foram checados individualmente
# Nao pensei der necessario inclui-los abaixo
checkDataRanges()

Zero valores faltantes
Todos os valores sao nao-negativos
Anos de contrucao entre 1900 e 2015
Ano de renovacao >= ano de contrucao
Ano de renovacao <= 2015


## Avaliação

#### Gerar exemplo da pipeline de Corretoria Automatizada

---

O exemplo será o dado no relatório:

<cite> 
Imagine-se que um cliente busca um imóvel com preço entre 500 mil e 700 mil dólares, que seja novo, e possua boas condições gerais de uso (bem cuidado).
</cite>

Os passos são:

1. Entrevista com o cliente (realizada acima)
2. Consulta no banco de dados
3. Aplicação de k-means
4. Análise heurística

In [8]:
from pysrc.eval import *

#### Passo 2 -- Consulta no banco de dados

In [9]:
reducedDataframe = dataframe.query("price >= 500000 & price <= 700000 & " +
                                   "yr_built >= 2000 & " +
                                   "condition >= 4"
                                  )
reducedDataframe

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
1326,625000,3,3.50,1810,1846,2.0,0,0,4,8,1440,370,2009,2009,1480,3600
8221,506000,5,3.75,3880,8370,2.0,0,0,4,7,3880,0,2004,2004,2160,4651
10682,525000,4,2.50,2310,5573,2.0,0,0,4,9,2310,0,2003,2003,2310,6189
14106,600000,5,3.25,4410,58157,2.0,0,0,4,9,2330,2080,2001,2001,2460,42565
14498,635000,2,2.50,2470,8840,2.0,0,0,4,8,1780,690,2001,2001,1940,8840
18222,529999,3,2.50,1710,1664,2.0,0,0,5,8,1300,410,2003,2003,1470,5400
21212,552100,3,3.00,1330,1379,2.0,0,0,4,8,1120,210,2005,2005,1810,1770


#### Passo 3 -- Aplicação de k-means

In [10]:
# Retorna os centroides dos clusters para o numero ideal de grupos K
# Podemos escolher o número máximo de clusters maxK
# Como só temos 6 instâncias na tabela acima, convém que o máximo seja 3 grupos
reps = getRepresentatives(reducedDataframe, maxK=3)


Calculated gap statistic:  0.24124321619563638

Calculated gap statistic:  -0.24656399446964627


In [11]:
reps

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,528274.75,3.750000,2.937500,2307.500000,4246.500000,2.0,0.0,0.0,4.25,8.000000,2152.5,155.000000,2003.750000,2003.750000,1937.5,4502.5
1,620000.00,3.333333,3.083333,2896.666667,22947.666667,2.0,0.0,0.0,4.00,8.333333,1850.0,1046.666667,2003.666667,2003.666667,1960.0,18335.0


#### Passo 4 -- Análise heurística

O número de instâncias depois de realizar a consulta é muito pequeno. Talvez um outro método possa resolver isso, ou deve-se simplesmente analisar a tabela original. Neste caso, o processo não se mostrou construtivo, apesar de termos conseguido um agrupamento com a estatística gap positiva.

Uma boa ideia pode ser estabelecer um limiar mínimo, como 20 imóveis, a partir do qual se utiliza a Corretoria Automatizada. Isso seria decidido empiricamente, através do uso repetido do tal método.